In [1]:
import requests 
from bs4 import BeautifulSoup

import time
import random as rand 

import pandas as pd

review_dict_final = {'name':[], 'rating':[], 'review_count':[]}

review_urls = ['https://www.metacritic.com/game/pc/halo-infinite/user-reviews?sort-by=most-helpful&num_items=100&page=0', 'https://www.metacritic.com/game/playstation-5/call-of-duty-modern-warfare-ii/user-reviews', 'https://www.metacritic.com/game/pc/doom/user-reviews', 'https://www.metacritic.com/game/pc/battlefield-2042/user-reviews', 'https://www.metacritic.com/game/pc/destiny-2/user-reviews', 'https://www.metacritic.com/game/pc/deep-rock-galactic/user-reviews', 'https://www.metacritic.com/game/pc/tom-clancys-rainbow-six-siege/user-reviews', 'https://www.metacritic.com/game/pc/apex-legends/user-reviews', 'https://www.metacritic.com/game/pc/ready-or-not/user-reviews', 'https://www.metacritic.com/game/pc/payday-2/user-reviews']

for u in review_urls:
    url = u
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response  = requests.get(url, headers = user_agent)
    time.sleep(rand.randint(3,30)) 
    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', id='main_content'):
        review_dict_final['name'].append(review.find('div', class_='product_title').find('h1').text)
        review_dict_final['rating'].append(float(review.find('div', class_='userscore_wrap feature_userscore').find_all('div')[1].text))
        count_string = review.find('div', class_='summary').find_all('strong')[0].text
        count = int(count_string.split()[0])
        review_dict_final['review_count'].append(count)

reviews = pd.DataFrame(review_dict_final)

In [2]:
reviews

,name,rating,review_count
0,Halo Infinite,6.6,1145
1,Call of Duty: Modern Warfare II,5.0,516
2,DOOM,8.5,3579
3,Battlefield 2042,2.1,4749
4,Destiny 2,4.3,1679
5,Deep Rock Galactic,8.2,306
6,Tom Clancy's Rainbow Six Siege,7.0,1814
7,Apex Legends,6.1,1655
8,Ready or Not,8.6,14
9,Payday 2,3.9,3925


In [8]:
player_count_dict = {'game_name':[], '24-count':[]}

player_count_ids = [1240440, 1938090, 379720, 1517290, 1085660, 548430, 359550, 1172470, 1144200, 218620]

for i in player_count_ids:
    url = 'https://steamcharts.com/app/'+str(i)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response  = requests.get(url, headers = user_agent)
    time.sleep(rand.randint(3,30)) 
    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', id='content-wrapper'):
        if review.find('h1', id='app-title') == None:
            print("broke")
            break 
        player_count_dict['game_name'].append(review.find('h1', id='app-title').find('a').text)
        player_count_dict['24-count'].append(int(review.find_all('div', class_='app-stat')[1].find('span', class_='num').text))

counts = pd.DataFrame(player_count_dict)

In [9]:
counts

,game_name,24-count
0,Halo Infinite,6849
1,Call of Duty®: Modern Warfare® II | Warzone™ 2.0,181828
2,DOOM,1112
3,Battlefield™ 2042,9672
4,Destiny 2,86070
5,Deep Rock Galactic,22431
6,Tom Clancy's Rainbow Six Siege,53647
7,Apex Legends,304241
8,Ready or Not,6991
9,PAYDAY 2,44612


In [10]:
combined = pd.concat([reviews, counts], axis=1)
combined

,name,rating,review_count,game_name,24-count
0,Halo Infinite,6.6,1145,Halo Infinite,6849
1,Call of Duty: Modern Warfare II,5.0,516,Call of Duty®: Modern Warfare® II | Warzone™ 2.0,181828
2,DOOM,8.5,3579,DOOM,1112
3,Battlefield 2042,2.1,4749,Battlefield™ 2042,9672
4,Destiny 2,4.3,1679,Destiny 2,86070
5,Deep Rock Galactic,8.2,306,Deep Rock Galactic,22431
6,Tom Clancy's Rainbow Six Siege,7.0,1814,Tom Clancy's Rainbow Six Siege,53647
7,Apex Legends,6.1,1655,Apex Legends,304241
8,Ready or Not,8.6,14,Ready or Not,6991
9,Payday 2,3.9,3925,PAYDAY 2,44612


In [13]:
combined_clean = combined.drop(columns=['game_name'])
combined_clean

,name,rating,review_count,24-count
0,Halo Infinite,6.6,1145,6849
1,Call of Duty: Modern Warfare II,5.0,516,181828
2,DOOM,8.5,3579,1112
3,Battlefield 2042,2.1,4749,9672
4,Destiny 2,4.3,1679,86070
5,Deep Rock Galactic,8.2,306,22431
6,Tom Clancy's Rainbow Six Siege,7.0,1814,53647
7,Apex Legends,6.1,1655,304241
8,Ready or Not,8.6,14,6991
9,Payday 2,3.9,3925,44612


In [14]:
combined_clean.to_csv('base_data.csv', index=False)